In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import time
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import cross_validate

In [2]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

## Two datasets

1. Original unencoded. We would scale and encode in our own methods.

2. Encoded numerical. We would use it as a comparison.

In [3]:
df_original = pd.read_csv("Drug_Consumption.csv")
df_quantified = pd.read_csv("Drug_Consumption_Quantified.csv")

print("Shape of original dataset: ", df_original.shape)
print("Shape of quantified dataset: ", df_quantified.shape)

Shape of original dataset:  (1884, 32)
Shape of quantified dataset:  (1884, 32)


## Quantified dataset preprocess

In [5]:
df = df_quantified.drop(['ID'],axis=1)
df = df[df['Semer']=='CL0']
df = df.drop(['Semer'],axis=1)
print(df.shape)

(1876, 30)


In [6]:
drugs_to_ignore = ['Amphet', 'Amyl', 'Benzos', 'Ketamine', 'Legalh', 'Meth', 'VSA']
drugs_to_consider = ['Alcohol', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'LSD', 'Mushrooms', 'Nicotine']

In [7]:
non_alcohol_feature = drugs_to_ignore + ['Caff', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'LSD', 'Mushrooms', 'Nicotine']
non_caff_features = drugs_to_ignore + ['Alcohol', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'LSD', 'Mushrooms', 'Nicotine']
non_cannabis_features = drugs_to_ignore + ['Alcohol', 'Caff', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'LSD', 'Mushrooms', 'Nicotine']
non_choc_features = drugs_to_ignore + ['Alcohol', 'Caff', 'Cannabis', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'LSD', 'Mushrooms', 'Nicotine']
non_coke_features = drugs_to_ignore + ['Alcohol', 'Caff', 'Cannabis', 'Choc', 'Crack', 'Ecstasy', 'Heroin', 'LSD', 'Mushrooms', 'Nicotine']
non_crack_features = drugs_to_ignore + ['Alcohol', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Ecstasy', 'Heroin', 'LSD', 'Mushrooms', 'Nicotine']
non_ecstasy_features = drugs_to_ignore + ['Alcohol', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Heroin', 'LSD', 'Mushrooms', 'Nicotine']
non_heroin_features = drugs_to_ignore + ['Alcohol', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'LSD', 'Mushrooms', 'Nicotine']
non_lsd_features = drugs_to_ignore + ['Alcohol', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'Mushrooms', 'Nicotine']
non_mushrooms_features = drugs_to_ignore + ['Alcohol', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'LSD', 'Nicotine']
non_nicotine_features = drugs_to_ignore + ['Alcohol', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'LSD', 'Mushrooms']

In [8]:
drug_enc=OrdinalEncoder(categories=[["CL0","CL1","CL2","CL3","CL4","CL5","CL6"]])
for feature in drugs_to_consider:
    df[feature]=drug_enc.fit_transform(df[feature].to_numpy().reshape(-1,1))

In [9]:
df_quan_alcohol = df.drop(non_alcohol_feature, axis = 1)

df_quan_caff = df.drop(non_caff_features, axis = 1)

df_quan_cannabis = df.drop(non_cannabis_features, axis = 1)

df_quan_choc = df.drop(non_choc_features, axis = 1)

df_quan_coke = df.drop(non_coke_features, axis = 1)

df_quan_crack = df.drop(non_crack_features, axis = 1)

df_quan_ecstasy = df.drop(non_ecstasy_features, axis = 1)

df_quan_heroin = df.drop(non_heroin_features, axis = 1)

df_quan_lsd = df.drop(non_lsd_features, axis = 1)

df_quan_mushrooms = df.drop(non_mushrooms_features, axis = 1)

df_quan_nicotine = df.drop(non_nicotine_features, axis = 1)

In [10]:
df_quan_nicotine.head()

,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,AScore,Cscore,Impulsive,SS,Nicotine
0,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,4.0
1,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,0.0
2,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,2.0
3,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,2.0
4,2.59171,0.48246,-1.22751,0.24923,-0.31685,-0.67825,-0.30033,-1.55521,2.03972,1.63088,-1.37983,-1.54858,6.0


## Original dataset preprocess

In [11]:
df1 = df_original.drop(['ID'],axis=1)
df1 = df1[df1['Semer']=='CL0']
df1 = df1.drop(['Semer'],axis=1)
df1 = df1.drop(drugs_to_ignore,axis=1)
df1.shape

(1876, 23)

In [12]:
numerical_feature = ['Nscore','Escore','Oscore','AScore','Cscore','Impulsive','SS']
nominal_cat_feature=["Gender","Country","Ethnicity"] ##apply onehot encoder to encode nominal categorical features

## apply ordinalecoder to encode ordinal categorical features and target values.
age_enc=OrdinalEncoder(categories=[["18-24","25-34","35-44","45-54","55-64","65+"]])
edu_enc=OrdinalEncoder(categories=[["Left school before 16 years","Left school at 16 years","Left school at 17 years","Left school at 18 years","Some college or university, no certificate or degree","Professional certificate/ diploma","University degree","Masters degree","Doctorate degree"]])
drug_enc=OrdinalEncoder(categories=[["CL0","CL1","CL2","CL3","CL4","CL5","CL6"]])

## ordinal encoder seems not encode multiple varibles at same time, so encode targets values one by one first.
for feature in drugs_to_consider:
    df1[feature] = drug_enc.fit_transform(df1[feature].to_numpy().reshape(-1,1))

preprocess=make_column_transformer((StandardScaler(),numerical_feature),
                                   (age_enc,["Age"]),
                                   (edu_enc, ["Education"]),
                                   (OneHotEncoder(handle_unknown="ignore",drop='first'),nominal_cat_feature),
                                   remainder="passthrough")
df_trans = preprocess.fit_transform(df1)

In [13]:
df_trans.shape

(1876, 33)

In [14]:
df_ori_X = df_trans[:,:22]
df_ori_X.shape

(1876, 22)

In [15]:
df_ori_alcohol = np.c_[df_ori_X,df_trans[:,22]]

df_ori_caff = np.c_[df_ori_X,df_trans[:,23]]

df_ori_cannabis = np.c_[df_ori_X,df_trans[:,24]]

df_ori_choc = np.c_[df_ori_X,df_trans[:,25]]

df_ori_coke = np.c_[df_ori_X,df_trans[:,26]]

df_ori_crack = np.c_[df_ori_X,df_trans[:,27]]

df_ori_ecstasy = np.c_[df_ori_X,df_trans[:,28]]

df_ori_heroin = np.c_[df_ori_X,df_trans[:,29]]

df_ori_lsd = np.c_[df_ori_X,df_trans[:,30]]

df_ori_mushrooms = np.c_[df_ori_X,df_trans[:,31]]

df_ori_nicotine = np.c_[df_ori_X,df_trans[:,32]]

In [16]:
df_ori_alcohol.shape

(1876, 23)

## modelling

In [24]:
def split(df_X, df_Y):
    X_dev, X_test, y_dev, y_test = train_test_split(df_X, df_Y, test_size=0.2, random_state=42)
    return X_dev,X_test,y_dev,y_test

In [25]:
quantified_df = [df_quan_alcohol, df_quan_caff, df_quan_cannabis, df_quan_choc, df_quan_coke, df_quan_crack,\
                  df_quan_ecstasy, df_quan_heroin, df_quan_lsd, df_quan_mushrooms, df_quan_nicotine]
original_df = [df_ori_alcohol, df_ori_caff, df_ori_cannabis, df_ori_choc, df_ori_coke, df_ori_crack,\
                  df_ori_ecstasy, df_ori_heroin, df_ori_lsd, df_ori_mushrooms, df_ori_nicotine]

## Baseline Decision Tree Classifier

In [36]:
for i in range(11):
    print(drugs_to_consider[i])
    quan_df = quantified_df[i]
    ori_df = original_df[i]
    quan_df_X = quan_df.drop(columns=[drugs_to_consider[i]])
    quan_df_y = quan_df[drugs_to_consider[i]]
    ori_df_X = ori_df[:,:22]
    ori_df_y = ori_df[:,-1]
    X_dev_quan,X_test_quan,y_dev_quan,y_test_quan = split(quan_df_X, quan_df_y)
    X_dev_ori,X_test_ori,y_dev_ori,y_test_ori = split(ori_df_X, ori_df_y)
    dtclf_quan = DecisionTreeClassifier(random_state = 0, max_depth = 10)
    dtclf_quan.fit(X_dev_quan,y_dev_quan)
    print("quantified dataset")
    print('train acc',dtclf_quan.score(X_dev_quan,y_dev_quan),'test acc',dtclf_quan.score(X_test_quan,y_test_quan))
    y_test_quan_pred = dtclf_quan.predict(X_test_quan)
    print('test confusion matrix\n',metrics.confusion_matrix(y_test_quan, y_test_quan_pred))
    dtclf_ori = DecisionTreeClassifier(random_state = 0, max_depth = 10)
    dtclf_ori.fit(X_dev_ori,y_dev_ori)
    print("original dataset")
    print('train acc',dtclf_ori.score(X_dev_ori,y_dev_ori),'test acc',dtclf_ori.score(X_test_ori,y_test_ori))
    y_test_ori_pred = dtclf_ori.predict(X_test_ori)
    print('test confusion matrix\n',metrics.confusion_matrix(y_test_ori, y_test_ori_pred))

Alcohol
quantified dataset
train acc 0.712 test acc 0.31648936170212766
test confusion matrix
 [[ 1  0  0  0  0  4  2]
 [ 0  1  0  0  0  4  2]
 [ 0  0  0  1  2  6  3]
 [ 2  1  2  2  7 17 11]
 [ 2  1  1  3 10 38  9]
 [ 2  2  7  9 15 82 35]
 [ 2  0  2  4  8 53 23]]
original dataset
train acc 0.694 test acc 0.3351063829787234
test confusion matrix
 [[ 1  0  0  0  0  5  1]
 [ 0  0  1  0  0  5  1]
 [ 0  1  0  0  1  6  4]
 [ 1  1  2  5 11 13  9]
 [ 0  1  0  5 11 32 15]
 [ 2  2  4 10 12 83 39]
 [ 0  0  1  4  7 54 26]]
Caff
quantified dataset
train acc 0.7853333333333333 test acc 0.6781914893617021
test confusion matrix
 [[  1   0   0   0   0   0   4]
 [  0   0   0   0   0   0   1]
 [  0   0   0   0   0   0   5]
 [  0   0   0   0   0   0  15]
 [  0   0   1   0   0   1  19]
 [  1   1   2   0   2  10  37]
 [  1   1   3   0   4  23 244]]
original dataset
train acc 0.832 test acc 0.651595744680851
test confusion matrix
 [[  0   0   0   0   0   0   5]
 [  0   0   0   0   0   0   1]
 [  0   0   0   